In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import json
import os

In [3]:
load_dotenv(override=True)

True

In [4]:
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key and openai_api_key[:8] == "sk-proj-":
    print("OPENAI_API_KEY is present")
else:
    print("OPENAI_API_KEY is not present")

OPENAI_API_KEY is present


In [5]:
openai = OpenAI()

In [6]:
def get_character_profile_information(background, notable_works, occupation, first_appearance, era):
    return {
        "background": background,
        "notable_works": notable_works,
        "occupation": occupation,
        "first_appearance": first_appearance,
        "era": era
    }

In [7]:
get_character_profile_information_json = {
    "name": "get_character_profile_information",
    "description": "Get the character's information like the background, notable works, occupation, first appearance, and era",
    "parameters": {
        "type": "object",
        "properties": {
            "background": {
                "type": "string",
                "description": "Background or the summary of information the character, everything should be factual, and not exceeding 1000 characters."
            },
            "notable_works": {
                "type": "string",
                "description": "A concise summary of the character’s most notable achievements, contributions, or works that made them significant in history or fiction.",
                "example": "Theory of Special Relativity, Theory of General Relativity, Photoelectric Effect, Mass-Energy Equivalence (E=mc²)"
            },
            "occupation": {
                "type": "string",
                "description": "The primary profession, role, or title of the character.",
                "example": "Theoretical Physicist"
            },
            "first_appearance": {
                "type": "string",
                "description": "The date, year, or title of the work/event where the character first appeared or was introduced. This can be a publication, film, historical record, or other medium.",
                "example": "Action Comics #1 (1938)"
            },
            "era": {
                "type": "string",
                "description": "The historical or fictional time period in which the character lived or was most active, including the range of years. Strictly follow the format in the example.",
                "example": "19th Century (1879–1955)"
            }
        },
        "required": ["background", "notable_works", "occupation", "first_appearance", "era"],
        "additionalProperties": False
    }
}

In [8]:
tools = [{"type": "function", "function": get_character_profile_information_json}]

In [9]:
model = "gpt-4o-mini"

In [10]:
character_name = "Albert Einstein"

In [11]:
messages = [
    {"role": "system", "content": "You are a narrator who knows a lot about people and providing character's information as if you really know them."},
    {"role": "user", "content": f"Give me a factual background information of {character_name}. If you don't know this character, just provide N/A and don't mess things up."}
]

In [12]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools
)

message = response.choices[0].message

print(message)


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_yMG6a10OmKITB8LvMU59PrG1', function=Function(arguments='{"background":"Albert Einstein was a theoretical physicist born on March 14, 1879, in Ulm, Germany. He is best known for developing the theory of relativity, which revolutionized our understanding of space, time, and gravity. Einstein\'s work laid the foundation for modern physics and introduced concepts such as mass-energy equivalence, encapsulated in the famous equation E=mc². During his lifetime, he made significant contributions to quantum theory, statistical mechanics, and cosmology.","notable_works":"His most notable works include the theory of relativity (Special Relativity in 1905, General Relativity in 1915) and the photoelectric effect, for which he received the Nobel Prize in Physics in 1921. His equation E=mc² is one of the most well-known equatio

In [13]:
def handle_tool_calls(tool_calls):
    results = {}
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        tool = globals().get(tool_name) if tool_name else { "error": "tool name is not found" }
        result = tool(**arguments)
        results = result
    return results

In [16]:
character_information = handle_tool_calls(message.tool_calls)

In [ ]:
character_system_message = (
    f"You are {character_name}. "
    f"Stay strictly in character, matching {character_name}'s identity, tone, knowledge, and personality. "
    f"Here is the initial information about {character_name}: {character_information}. "
    f"If the user greets or makes a casual remark, respond naturally as {character_name} without saying 'I don't know'. "
    f"If the user asks about something you truly have no information on, briefly say 'I don't know' "
    f"and optionally suggest a related topic you can discuss. "
    f"Be concise: at most 2–3 sentences unless the user explicitly asks for more detail."
)

narrator_system_message = (
    f"You are a factual narrator providing information about {character_name}. "
    f"Here is the initial information: {character_information}. "
    f"Answer accurately based on this information. "
    f"If the user greets or asks something general, reply politely and share a short fact or summary about {character_name}. "
    f"Only say 'I don't know' for specific facts that are not in the provided information. "
    f"Be concise: at most 2–3 sentences unless more detail is explicitly requested."
)

In [19]:
def chat(history, user_message, role):
    message = [{"role": "system", "content": character_system_message if role == character_name else narrator_system_message}] + history + [{ "role": "user", "content": user_message }]

    response = openai.chat.completions.create(
        model=model,
        messages=message
    )

    print(response.choices[0].message.content)

In [21]:
chat([], "Hi there", "Albert Einstein")

Hello! It is a pleasure to engage with you. How may I assist you on this fine day?
